In [1]:
import pandas as pd

ner = pd.read_csv('~/Downloads/ner_dataset.csv', encoding = 'cp1252')
ner

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [2]:
#Keep backup copy
ner_copy = ner.copy()

In [3]:
def ner_cleaner(df):
    
    #Fill missing sentence numbers forward
    df['Sentence #'] = df['Sentence #'].ffill()

    tag_dict = {'B-geo': 'PLACE', 
           'B-gpe': 'GPE',
           'B-per': 'PERSON', 
           'I-geo': 'PLACE',
           'B-org': 'ORG',
           'I-org': 'ORG',
           'B-tim': 'TIME',
           'I-per': 'PERSON',
           'I-gpe': 'GPE',
           'I-tim': 'TIME',
           'I-art': 'O',
           'B-art': 'O',
           'B-eve': 'O',
           'I-eve': 'O',
           'B-nat': 'O',
           'I-nat': 'O',    
          }
    #Map the dict to tag replacements
    df['Tag'] = df['Tag'].map(tag_dict).fillna(ner['Tag'])   
    
    #Transform sentence numbers to ints
    df['Sentence #'] = df['Sentence #'].apply(lambda x: int(x.split(': ')[1]))
    
    return df

ner = ner_cleaner(ner_copy)

In [4]:
def sentence_maker(df):
    sent_num = 1
    sents = []
    new_sent = []
    named_ents = []
    for ix, row in df.iterrows():
        
        if row['Sentence #']== sent_num:
            new_sent.append(row['Word'])
        else:
            #Add new sentence to list of sentences and reset
            new_sent = ' '.join([x for x in new_sent])
            sents.append(new_sent)
            #Begin next sentence with first new word
            new_sent = [row['Word']]
            sent_num+=1
    
    sent_df = pd.DataFrame(sents, columns = ['Text'])
            
    return sent_df
        
results = sentence_maker(ner)

In [ ]:
def entity_labeler(row_num):
    named_ents = []
    sent_slice = ner[ner['Sentence #'] == row_num + 1]

    for ix, row in sent_slice.iterrows():
        if row['Tag'] != 'O':
            #Find named entities and label their start/end indices with tag
            start_ix = results.iloc[row_num]['Text'].find(row['Word'])
            named_ents.append((start_ix, start_ix + len(row['Word']), row['Tag']))
            
    named_ents = {'entities': named_ents}
    
    return named_ents

new_ents = pd.Series(map(entity_labeler, range(len(results))))


In [ ]:
results = pd.concat([results, new_ents.rename('Entities')], axis=1)

In [ ]:
results